In [23]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [24]:
tf.config.list_physical_devices('GPU')

[]

In [25]:
import tensorflow
tensorflow.test.is_gpu_available()

False

In [26]:
batchsize=8

In [27]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'C:\Users\Saranya Katare\Downloads\mrlEyes_2018_01\train data',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'C:\Users\Saranya Katare\Downloads\mrlEyes_2018_01\train data',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 60239 images belonging to 2 classes.
Found 15059 images belonging to 2 classes.


In [28]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'C:\Users\Saranya Katare\Downloads\mrlEyes_2018_01\test data',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 9600 images belonging to 2 classes.


In [29]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

87910968/87910968 [==============================] - 23s 0us/step


In [30]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d_82 (Conv2D)             (None, 39, 39, 32)   864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_82 (BatchN  (None, 39, 39, 32)  96          ['conv2d_82[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_81 (Activation)     (None, 39, 39, 32)   0           ['batch_normalization_82[0][0

In [34]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [35]:
checkpoint = ModelCheckpoint(r'C:\Users\Saranya Katare\Downloads\mrlEyes_2018_01\models\model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [36]:
checkpoint = ModelCheckpoint(r'C:\Users\Saranya Katare\Downloads\mrlEyes_2018_01\models\model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

C:\Users\Saranya Katare\AppData\Local\Temp\ipykernel_5404\3918530525.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


Epoch 1/5
7529/7529 [==============================] - ETA: 0s - loss: 0.1762 - accuracy: 0.9330
Epoch 1: val_loss improved from inf to 0.21402, saving model to C:\Users\Saranya Katare\Downloads\mrlEyes_2018_01\models\model.h5
7529/7529 [==============================] - 807s 105ms/step - loss: 0.1762 - accuracy: 0.9330 - val_loss: 0.2140 - val_accuracy: 0.9099 - lr: 0.0010
Epoch 2/5
7529/7529 [==============================] - ETA: 0s - loss: 0.1658 - accuracy: 0.9364
Epoch 2: val_loss did not improve from 0.21402
7529/7529 [==============================] - 536s 71ms/step - loss: 0.1658 - accuracy: 0.9364 - val_loss: 0.2504 - val_accuracy: 0.8974 - lr: 0.0010
Epoch 3/5
7529/7529 [==============================] - ETA: 0s - loss: 0.1620 - accuracy: 0.9381
Epoch 3: val_loss did not improve from 0.21402
7529/7529 [==============================] - 6465s 859ms/step - loss: 0.1620 - accuracy: 0.9381 - val_loss: 0.2815 - val_accuracy: 0.8860 - lr: 0.0010
Epoch 4/5
7529/7529 [==============

In [37]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

C:\Users\Saranya Katare\AppData\Local\Temp\ipykernel_5404\713691994.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_tr, loss_tr = model.evaluate_generator(train_data)


0.12848463654518127
0.9506300091743469


In [38]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

C:\Users\Saranya Katare\AppData\Local\Temp\ipykernel_5404\4081756742.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr, loss_vr = model.evaluate_generator(validation_data)


0.22557049989700317
0.9135400652885437


In [39]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)

C:\Users\Saranya Katare\AppData\Local\Temp\ipykernel_5404\3655471885.py:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_test, loss_test = model.evaluate_generator(test_data)


0.12848463654518127
0.9506300091743469
